In [117]:
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score


In [118]:
ss_fin_module = joblib.load('ss_fin_30.pkl')
fin_module30 = joblib.load( 'fin_model_no_woe30_15_best.pkl')

In [119]:
df_2021 = pd.read_csv('counterparty-analysis/agents2021.csv')
df_2021 = df_2021.drop(['Unnamed: 0'], axis = 1)

In [120]:
df_2021['default0'] = ((df_2021['ПДЗ 31-90']+ df_2021['ПДЗ 91-365']+
                                                    df_2021['ПДЗ более 365'])>0).astype(int)

In [121]:
import columns_mapping as cm

In [122]:
df_2021 = cm.rename_columns(df_2021, 2021)

In [140]:
df_small = df_2021[['кратк_обязат_lag2','выручка_lag1','кредит_задол_lag1', 'немат_активы_lag1',
                   'прибыль_lag2', 'немат_активы_lag2', 'заемн_ср_кратк_lag3', 'внеоб_активы_lag3', 'долгоср_обязат_lag1',
                   'ук_lag3', 'заемн_ср_кратк_lag2', 'прибыль_до_налога_lag2', 'прибыль_lag3']]

In [124]:
df_small['div_log_кратк_обязат_lag2_log_выручка_lag1'] = np.log1p(df_small['кратк_обязат_lag2'])/ \
    np.log1p(df_small['выручка_lag1'])
df_small['mult_кредит_задол_lag1_кратк_обязат_lag2'] = df_small['кредит_задол_lag1']* df_small['кратк_обязат_lag2']
df_small['div_немат_активы_lag1_заемн_ср_кратк_lag2'] = df_small['немат_активы_lag1']/ df_small['заемн_ср_кратк_lag2']
df_small['div_немат_активы_lag1_прибыль_lag2'] = df_small['немат_активы_lag1']* df_small['прибыль_lag2']
df_small['div_немат_активы_lag2_заемн_ср_кратк_lag3'] = df_small['немат_активы_lag2']/ df_small['заемн_ср_кратк_lag3']
df_small['div_внеоб_активы_lag3_долгоср_обязат_lag1'] = df_small['внеоб_активы_lag3']* df_small['долгоср_обязат_lag1']
df_small['div_ук_lag3_заемн_ср_кратк_lag2'] = df_small['ук_lag3']/ df_small['заемн_ср_кратк_lag2']
df_small['div_прибыль_до_налога_lag2_прибыль_lag3'] = df_small['прибыль_до_налога_lag2']/ df_small['прибыль_lag3']


/var/folders/l9/x43hsk7d2x1g2r7qw4f9nbg40000gn/T/ipykernel_77700/2894406479.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['div_log_кратк_обязат_lag2_log_выручка_lag1'] = np.log1p(df_small['кратк_обязат_lag2'])/ \
/var/folders/l9/x43hsk7d2x1g2r7qw4f9nbg40000gn/T/ipykernel_77700/2894406479.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['mult_кредит_задол_lag1_кратк_обязат_lag2'] = df_small['кредит_задол_lag1']* df_small['кратк_обязат_lag2']
/var/folders/l9/x43hsk7d2x1g2r7qw4f9nbg4

In [125]:
features = ['div_немат_активы_lag1_заемн_ср_кратк_lag2',
 'mult_кредит_задол_lag1_кратк_обязат_lag2',
 'div_прибыль_до_налога_lag2_прибыль_lag3',
 'div_немат_активы_lag1_прибыль_lag2',
 'div_ук_lag3_заемн_ср_кратк_lag2',
 'div_внеоб_активы_lag3_долгоср_обязат_lag1',
 'div_немат_активы_lag2_заемн_ср_кратк_lag3',
 'div_log_кратк_обязат_lag2_log_выручка_lag1']

In [126]:
df_small = df_small.fillna(0)
df_small[df_small.isin([np.inf, -np.inf])] = 0 # заменим нулем

In [127]:
df_small_ss = df_small[features].copy()

In [128]:
df_small_ss[features] = ss_fin_module.transform(df_small_ss[features])

In [135]:
df_2021['preds_test'] = fin_module30.predict_proba(df_small_ss[features])[:,1]